Author: Rashad Haddad  
Description: Compute scores for example JSONs.  Data is relevant as of Mar 3rd, 2023

In [1]:
import pandas as pd
import numpy as np

import json
import sys
if "../" not in sys.path:
    sys.path.append("../")
from lib.compute_score import compute_score

In [2]:
%load_ext autoreload

In [3]:
def make_report(dat):
    # instantiate obligor
    obl = compute_score(input_data=dat, start_alpha=10, start_beta=10, protocol_name="aave_v3")
    
    # get counts of each transaction type for reference...
    counts = {'borrow': 0, 'repay': 0, 'liquidation': 0, 'deposit': 0, 'withdraw': 0}
    min_timestamp = np.inf
    max_timestamp = 0
    for tx in dat:
        counts[tx['type']] = counts.get(tx['type'],0) + 1

        if int(tx['timestamp']) > max_timestamp:
            max_timestamp = int(tx["timestamp"])
        if int(tx['timestamp']) < min_timestamp:
            min_timestamp = int(tx["timestamp"])

    # get report
    print("Number of transactions --> ", len(dat))
    print("Number of borrows --> ", counts['borrow'])
    print("Number of repays --> ", counts['repay'])
    print("Number of liquidates --> ", counts['liquidation'])
    print("Number of deposits --> ", counts['deposit'])
    print("Number of withdraws --> ", counts['withdraw'])
    print("Finishing alpha (good credit param) --> ", obl._alpha)
    print("Finishing beta (bad credit param) --> ", obl._beta)
    print("Finishing score --> ", obl.get_score())
    print("Finishing 95% confidence interval --> ", obl.get_conf_interval())
    print("Earliest")

    return obl


### Example 1 0x2d3178af3dfbb679716cc14e245be0a9e594550

In [18]:
obl1 = make_report(json.load(open("../../../example_jsons/0x2d3178af3dfbb679716cc14e245be0a9e5945500.json","r")))


Number of transactions -->  52
Number of borrows -->  20
Number of repays -->  5
Number of liquidates -->  0
Number of deposits -->  25
Number of withdraws -->  2
Finishing alpha (good credit param) -->  18.623083681990213
Finishing beta (bad credit param) -->  18.139272255831564
Finishing score -->  51
Finishing 95% confidence interval -->  (34, 67)


### Example 2 0x9600a48ed0f931d0c422d574e3275a90d8b22745

In [16]:
%autoreload
obl2 = make_report(json.load(open("../../../example_jsons/0x9600a48ed0f931d0c422d574e3275a90d8b22745.json","r")))

Number of transactions -->  258
Number of borrows -->  84
Number of repays -->  24
Number of liquidates -->  0
Number of deposits -->  100
Number of withdraws -->  50
Finishing alpha (good credit param) -->  88.90587791937955
Finishing beta (bad credit param) -->  30.821560629873026
Finishing score -->  74
Finishing 95% confidence interval -->  (66, 82)


With a lower bound at 66, we are very confident this is a reliable borrower. This borrower can safely be given high efficiency terms.

### Example 3 0xa3fa88658d7f6ceea0288033e85de77d3c02f779 

In [6]:
obl3 = make_report(json.load(open("../../../example_jsons/0xa3fa88658d7f6ceea0288033e85de77d3c02f779.json","r")))

Number of transactions -->  8
Number of borrows -->  1
Number of repays -->  2
Number of liquidates -->  0
Number of deposits -->  3
Number of withdraws -->  2
Finishing alpha (good credit param) -->  16.53878759036552
Finishing beta (bad credit param) -->  10.460912222139333
Finishing score -->  61
Finishing 95% confidence interval -->  (43, 80)


## Example 4 0x9600a48ed0f931d0c422d574e3275a90d8b22745

In [7]:
obl4 = make_report(json.load(open("../../../example_jsons/0x9600a48ed0f931d0c422d574e3275a90d8b22745.json")))

Number of transactions -->  258
Number of borrows -->  84
Number of repays -->  24
Number of liquidates -->  0
Number of deposits -->  100
Number of withdraws -->  50
Finishing alpha (good credit param) -->  88.90587791937955
Finishing beta (bad credit param) -->  30.821560629873026
Finishing score -->  74
Finishing 95% confidence interval -->  (66, 82)


Despite the large tx volume the score here is likely low due to large amounts of credit being extended, without depositing large amounts of collateral. This borrower is likely to see a liquidation before repayment

### Example 5

In [8]:
obl5 = make_report(json.load(open("../../../example_jsons/0x6cfdad0772edc2ea15076a9706ec7c1f0a791ac1.json")))

Number of transactions -->  38
Number of borrows -->  11
Number of repays -->  9
Number of liquidates -->  0
Number of deposits -->  7
Number of withdraws -->  11
Finishing alpha (good credit param) -->  24.027202333170525
Finishing beta (bad credit param) -->  14.711665937064263
Finishing score -->  62
Finishing 95% confidence interval -->  (47, 77)


## Example 6 

In [9]:
%autoreload
obl6 = make_report(json.load(open("../../../example_jsons/0xbec69dfce4c1fa8b7843fee1ca85788d84a86b06.json","r")))

aEthUSDC ETH
Number of transactions -->  36
Number of borrows -->  13
Number of repays -->  4
Number of liquidates -->  1
Number of deposits -->  14
Number of withdraws -->  4
Finishing alpha (good credit param) -->  16.24790895189583
Finishing beta (bad credit param) -->  15.677803421766086
Finishing score -->  51
Finishing 95% confidence interval -->  (33, 68)


Depsite otherwise good history, we see that given this account has experienced a liquidation, the score is much lower...

### Example 7 0x23db246031fd6f4e81b0814e9c1dc0901a18da2d

This is an interesting example as there are multiple borrows against the same collat, and a liquidation.

In [10]:
dat7 = pd.json_normalize(json.load(open("../../../example_jsons/0x23db246031fd6f4e81b0814e9c1dc0901a18da2d.json","r")))

In [11]:
dat7

,amount,amountUSD,timestamp,logIndex,symbol,type
0,3.076774,4937.606882,1674866891,279,WETH,deposit
1,2530.580418,2530.508979,1674866891,284,DAI,borrow
2,905.720373,905.678927,1674866891,290,USDC,borrow
3,500.000000,500.100910,1675044023,130,USDC,borrow
4,0.857464,1322.616413,1675105907,154,aEthDAI,liquidation


In [12]:
%autoreload
obl7 = make_report(json.load(open("../../../example_jsons/0x23db246031fd6f4e81b0814e9c1dc0901a18da2d.json","r")))

aEthDAI ETH
Number of transactions -->  5
Number of borrows -->  3
Number of repays -->  0
Number of liquidates -->  1
Number of deposits -->  1
Number of withdraws -->  0
Finishing alpha (good credit param) -->  11.38254953631871
Finishing beta (bad credit param) -->  11.407437263944388
Finishing score -->  50
Finishing 95% confidence interval -->  (29, 70)


Given no repays and a liquidation, a score of 50 is fitting, the range however is very wide here at 29,70, indicating we have a small amount of information on this borrower. A conservative lender should demand high collateral ratios from this borrower.  

### Example 8 0x69bfc96c3983053c36e790c7eb79372bf6979518

In [13]:
obl8 = make_report(json.load(open("../../../example_jsons/0x69bfc96c3983053c36e790c7eb79372bf6979518.json","r")))

aEthUSDC ETH
Number of transactions -->  3
Number of borrows -->  1
Number of repays -->  0
Number of liquidates -->  1
Number of deposits -->  1
Number of withdraws -->  0
Finishing alpha (good credit param) -->  11.38254953631871
Finishing beta (bad credit param) -->  10.473700792228422
Finishing score -->  52
Finishing 95% confidence interval -->  (31, 73)


The score benefits here (slighly) from the deposit, but the range is wide.

### Example 9 0x2e18e31801870d91114fbfd33c6e4242242bec20

In [14]:
%autoreload
obl9 = make_report(json.load(open("../../../example_jsons/0x2e18e31801870d91114fbfd33c6e4242242bec20.json")))

Number of transactions -->  30
Number of borrows -->  11
Number of repays -->  3
Number of liquidates -->  0
Number of deposits -->  16
Number of withdraws -->  0
Finishing alpha (good credit param) -->  16.338414720618317
Finishing beta (bad credit param) -->  14.791268807880426
Finishing score -->  52
Finishing 95% confidence interval -->  (35, 70)


### Example 10  -- 0x2068d14ee50772dd6e89c6f64f32adad0cc936ef

This exampple is interesting due to as of the time it was downloaded there were 19 open positions

In [15]:
obl10 = make_report(json.load(open("../../../example_jsons/0x2068d14ee50772dd6e89c6f64f32adad0cc936ef.json")))

Number of transactions -->  20
Number of borrows -->  10
Number of repays -->  0
Number of liquidates -->  0
Number of deposits -->  10
Number of withdraws -->  0
Finishing alpha (good credit param) -->  22.05913668696993
Finishing beta (bad credit param) -->  14.213635371118105
Finishing score -->  61
Finishing 95% confidence interval -->  (45, 77)
